Copyright (c) 2023 Habana Labs, Ltd. an Intel Company.

# Summarization using T5 model from Hugging Face on the Optimum Habana Library

### Summarization with T5-3B model on the Intel&reg; Gaudi&reg; 2 AI acclerator
We will use the Hugging Face Summariazion example with the T5-3B model to fine tune the model with the CNN-dailymail dataset

run_summarization.py is a lightweight example of how to download and preprocess a dataset from the 🤗 Datasets library 

#### Initial Setup
We start with a Intel Gaudi PyTorch Docker image and run this notebook

#### Install the Intel Gaudi DeepSpeed Fork
The Intel Gaudi DeepSpeed Fork has implementations specifically for Gaudi and must be used

In [1]:
!pip install git+https://github.com/HabanaAI/DeepSpeed.git@1.16.2  

  Cloning https://github.com/HabanaAI/DeepSpeed.git (to revision 1.13.0) to /tmp/pip-req-build-h7p9s_le
  Running command git clone --filter=blob:none --quiet https://github.com/HabanaAI/DeepSpeed.git /tmp/pip-req-build-h7p9s_le
  Running command git checkout -b 1.13.0 --track origin/1.13.0
  Switched to a new branch '1.13.0'
  Branch '1.13.0' set up to track remote branch '1.13.0' from 'origin'.
  Resolved https://github.com/HabanaAI/DeepSpeed.git to commit 6522014efac08fdcbc37ea4a9d85552ce9cb7b50
  Preparing metadata (setup.py) ... done


#### Install the Optimum Habana Library

In [2]:
!pip install optimum-habana==v1.12.0
!pip install ipywidgets

#### Clone the Hugging Face Model Repository

In [4]:
!git clone -b v1.12.0 https://github.com/huggingface/optimum-habana.git

Cloning into 'optimum-habana'...
remote: Enumerating objects: 8392, done.
remote: Counting objects: 100% (3154/3154), done.
remote: Compressing objects: 100% (658/658), done.
remote: Total 8392 (delta 2795), reused 2553 (delta 2482), pack-reused 5238
Receiving objects: 100% (8392/8392), 3.77 MiB | 19.21 MiB/s, done.
Resolving deltas: 100% (5631/5631), done.


#### Go the Summarization example model and install the requirements

In [5]:
%cd optimum-habana/examples/summarization

/root/Gaudi2-Workshop/LLM-Training/optimum-habana/examples/summarization


In [6]:
!pip install -q -r requirements.txt

### Setup for DeepSpeed
Since we are using DeepSpeed, we have to confirm that the model has been configured properly.  We look for the following:

* model, optimizer, ... = deepspeed.initialize(args=args, model=model, optimizer=optimizer, ...)
* deepspeed.init_distributed(dist_backend=“hccl”, init_method=init_method)
* Create a ds_config.json file to set the DS training parameters.

#### DeepSpeed Initialization
Look in deepspeed.py and we see the model being passed to the DeepSpeed engine

```
    import deepspeed
    from deepspeed.utils import logger as ds_logger

    model = trainer.model
    args = trainer.args
    ...

    kwargs = {
        "args": habana_args,
        "model": model,
        "model_parameters": model_parameters,
        "config_params": config,
        "optimizer": optimizer,
        "lr_scheduler": lr_scheduler,
    }

    deepspeedengine, optimizer, _, lr_scheduler = deepspeed.initialize(**kwargs)

```

#### DeepSpeed Distributed
Look in training_args.py and we see the DeepSpeed Distribution initialization

```
    from habana_frameworks.torch.distributed.hccl import initialize_distributed_hpu
    world_size, rank, self.local_rank = initialize_distributed_hpu()

    import deepspeed
    deepspeed.init_distributed(dist_backend="hccl", timeout=timedelta(seconds=self.ddp_timeout))
       logger.info("DeepSpeed is enabled.")
```

#### Create DeepSpeed Config file with ZeRO preferences
The ds_config.json file will configure the parameters to run DeepSpeed

In this case, we will run the ZeRO2 optimizer and BF16 mixed precision.

In [7]:
%pwd

'/root/Gaudi2-Workshop/LLM-Training/optimum-habana/examples/summarization'

In [8]:
%%sh
tee ./ds_config.json <<EOF
{
    "steps_per_print": 64,
    "train_batch_size": "auto",
    "train_micro_batch_size_per_gpu": "auto",
    "gradient_accumulation_steps": "auto",
    "bf16": {
        "enabled": true
    },
    "gradient_clipping": 1.0,
    "zero_optimization": {
        "stage": 2,
        "overlap_comm": false,
        "reduce_scatter": false,
        "contiguous_gradients": false
    }
}
EOF

{
    "steps_per_print": 64,
    "train_batch_size": "auto",
    "train_micro_batch_size_per_gpu": "auto",
    "gradient_accumulation_steps": "auto",
    "bf16": {
        "enabled": true
    },
    "gradient_clipping": 1.0,
    "zero_optimization": {
        "stage": 2,
        "overlap_comm": false,
        "reduce_scatter": false,
        "contiguous_gradients": false
    }
}


#### Fine Tuning T5-3b with the cnn_dailymail dataset
The T5-3b model is a large language model that was originally trained on the C4 dataset and in this case will be fined tuned on the [cnn_dailymail](https://huggingface.co/datasets/cnn_dailymail) dataset that is an English-language dataset containing just over 300k unique news articles as written by journalists at CNN and the Daily Mail.

For use of this model on Intel Gaudi First-Gen, users should update the model to "T5-large"

This is run by `gaudi_spawn.py`, a simple launcher script to collect arguments and send them to `distributed_runner.py` for training on multiple HPUs, which then calls the `run_summarization.py` model.

Notice the Habana specific commands to use here:

-- use_habana  - allows training to run on Intel Gaudi cards
-- use_hpu_graphs - reduces recompilation by replaying the graph  
-- gaudi_config_name Habana/t5 - mapping to Hugging Face T5 Model  

**Even though a Billion parameter T5 model can be used for Fine Tuning, this fine tuning still takes many hours to complete.  
For users that wish to execute the example Fine Tuning, they should modify the `model_name_or_path` to "t5-small", which takes about 30 minutes to complete.**


In [8]:
%pwd

'/root/Gaudi2-Workshop/LLM-Training/optimum-habana/examples/summarization'

In [9]:
!mkdir ft-summarization

In [9]:
!python ../gaudi_spawn.py \
--world_size 8 --use_deepspeed run_summarization.py \
--model_name_or_path t5-small \
--do_train \
--dataset_name cnn_dailymail \
--dataset_config '"3.0.0"' \
--source_prefix '"summarize: "' \
--output_dir ./ft-summarization \
--per_device_train_batch_size 4 \
--per_device_eval_batch_size 4 \
--overwrite_output_dir \
--predict_with_generate \
--use_habana \
--use_lazy_mode \
--use_hpu_graphs_for_training \
--gaudi_config_name Habana/t5 \
--ignore_pad_token_for_loss False \
--pad_to_max_length \
--save_strategy epoch \
--report_to none \
--throughput_warmup_steps 3 \
--deepspeed ./ds_config.json

DistributedRunner run(): command = deepspeed --num_nodes 1 --num_gpus 8 --no_local_rank run_summarization.py --model_name_or_path t5-small --do_train --dataset_name cnn_dailymail --dataset_config "3.0.0" --source_prefix "summarize: " --output_dir ./ft-summarization --per_device_train_batch_size 4 --per_device_eval_batch_size 4 --overwrite_output_dir --predict_with_generate --use_habana --use_lazy_mode --use_hpu_graphs_for_training --gaudi_config_name Habana/t5 --ignore_pad_token_for_loss False --pad_to_max_length --save_strategy epoch --report_to none --throughput_warmup_steps 3 --deepspeed ./ds_config.json
/usr/local/lib/python3.8/dist-packages/torch/distributed/distributed_c10d.py:252: UserWarning: Device capability of hccl unspecified, assuming `cpu` and `cuda`. Please specify it via the `devices` argument of `register_backend`.
  warnings.warn(
[2024-01-09 23:04:21,461] [INFO] [real_accelerator.py:175:get_accelerator] Setting ds_accelerator to hpu (auto detect)
[2024-01-09 23:04:23

### After fine tuning, let's look at the results
This fine tuned model has created the new `pytorch_model.bin` and the global_step.. folder contains the checkpoints that will be used in the infernece in the next section.


In [10]:
%cd ./ft-summarization

/root/Gaudi2-Workshop/LLM-Training/optimum-habana/examples/summarization/ft-summarization


In [11]:
%ls -al

total 217852
drwxr-xr-x 5 root root      4096 Jan  9 23:43 ./
drwxr-xr-x 4 root root      4096 Jan  9 23:10 ../
-rw-r--r-- 1 root root       314 Jan  9 23:43 all_results.json
drwxr-xr-x 3 root root      4096 Jan  9 23:32 checkpoint-17946/
drwxr-xr-x 3 root root      4096 Jan  9 23:43 checkpoint-26919/
drwxr-xr-x 3 root root      4096 Jan  9 23:21 checkpoint-8973/
-rw-r--r-- 1 root root      1503 Jan  9 23:43 config.json
-rw-r--r-- 1 root root       247 Jan  9 23:43 gaudi_config.json
-rw-r--r-- 1 root root       358 Jan  9 23:43 generation_config.json
-rw-r--r-- 1 root root 219754786 Jan  9 23:43 pytorch_model.bin
-rw-r--r-- 1 root root      1182 Jan  9 23:43 README.md
-rw-r--r-- 1 root root      2201 Jan  9 23:43 special_tokens_map.json
-rw-r--r-- 1 root root    791656 Jan  9 23:43 spiece.model
-rw-r--r-- 1 root root     20746 Jan  9 23:43 tokenizer_config.json
-rw-r--r-- 1 root root   2422388 Jan  9 23:43 tokenizer.json
-rw-r--r-- 1 root root     13724 Jan  9 23:43 trainer_state.json


#### Summarization using the Pipeline
Now we can run the summarization using Hugging Face Pipeline call with the fine tuned model.  In this case we will point to the model that we fine tuned.   Remember that if you used t5-small to do the Fine Tuning, be sure to change the `model_to_finetune` to "t5-small"

In [1]:
import torch
import habana_frameworks.torch

from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer

# Load model to fine-tune and its tokenizer
model_to_finetune = "t5-small"
model = AutoModelForSeq2SeqLM.from_pretrained(model_to_finetune)
tokenizer = AutoTokenizer.from_pretrained(model_to_finetune)

# Point to the ft-summarization folder with the fine-tuned model
path_to_local_model = "/root/Gaudi2-Workshop/LLM-Training/optimum-habana/examples/summarization/ft-summarization"

# Instantiate pipeline from local repo, if you did not run the fine tuning step above, you can change: model=model_to_finetune
pipe = pipeline(task="summarization", model=path_to_local_model, device="hpu", torch_dtype=torch.bfloat16, min_length=50, max_length=150)


#text_to_summarize = "summarize: Photosynthesis involves a series of complex reactions that take place within specialized organelles called chloroplasts in plant cells. It can be broadly divided into two stages: the light-dependent reactions and the light-independent reactions, also known as the Calvin cycle.  Light-Dependent Reactions: During the light-dependent reactions, chlorophyll pigments within the thylakoid membranes of the chloroplasts absorb light energy. This energy is utilized to split water molecules into oxygen, protons (H+), and electrons. Oxygen is released as a byproduct, while protons and electrons are transported through an electron transport chain, generating ATP (adenosine triphosphate) and NADPH (nicotinamide adenine dinucleotide phosphate).  Light-Independent Reactions (Calvin Cycle):  The ATP and NADPH produced in the light-dependent reactions are utilized in the Calvin cycle, which takes place in the stroma of the chloroplasts. In this cycle, carbon dioxide from the atmosphere combines with the stored energy in the form of ATP and NADPH to produce glucose. This glucose serves as a building block for other carbohydrates and organic compounds. Photosynthesis is a complex process that enables plants, algae, and some bacteria to convert light energy into chemical energy, facilitating the sustenance of life on Earth. It involves the interplay of light-dependent reactions, which generate ATP and NADPH, and the light-independent reactions or the Calvin cycle, which utilize the produced energy to fix carbon dioxide and produce glucose. Enhancing our understanding of photosynthesis and its underlying mechanisms holds the key to various applications, including improving crop yields, developing sustainable bioenergy sources, and addressing environmental challenges."
text_to_summarize = "summarize: Introduction: The Strategic Arms Limitation Talks II (SALT II) treaty, signed on June 18, 1979, between the United States and the Soviet Union, marked a significant milestone in nuclear arms control efforts during the Cold War era. Building upon its predecessor, SALT I, the treaty aimed to curb the arms race and reduce the risk of nuclear conflict between the superpowers. Key Provisions: SALT II encompassed several crucial provisions. It placed limits on strategic offensive arms, including intercontinental ballistic missiles (ICBMs), submarine-launched ballistic missiles (SLBMs), and heavy bombers. The agreement specified the maximum number of deployed warheads and launchers each party could possess. Verification and Compliance: To ensure compliance, the treaty established comprehensive verification measures. This involved regular exchanges of data, on-site inspections, and monitoring activities by both nations. These measures sought to enhance transparency, foster trust, and prevent either side from gaining a significant advantage in terms of strategic nuclear capabilities. Ratification and Challenges: Although both the United States and the Soviet Union signed the treaty, its ratification faced considerable challenges. The political landscape changed when the Soviet Union invaded Afghanistan in 1979, leading to a deterioration of U.S.-Soviet relations. As a result, the United States never ratified the treaty formally, rendering it non-binding. However, both nations pledged to adhere to its principles, effectively implementing its provisions on a voluntary basis. Legacy and Impact: Despite the treaty's non-ratification, SALT II's legacy and impact were significant. It set the stage for subsequent arms control negotiations, providing a framework for future agreements such as the Intermediate-Range Nuclear Forces (INF) Treaty and the Strategic Arms Reduction Treaty (START). SALT II demonstrated the potential for cooperation between the superpowers and laid the groundwork for continued dialogue aimed at reducing the nuclear threat globally."
print("------------------------------------------------------------")
print("Input:", text_to_summarize)
print()

result = pipe(text_to_summarize)
print("------------------------------------------------------------")
print("Result:", result)



/usr/local/lib/python3.8/dist-packages/torch/distributed/distributed_c10d.py:252: UserWarning: Device capability of hccl unspecified, assuming `cpu` and `cuda`. Please specify it via the `devices` argument of `register_backend`.
  warnings.warn(
============================= HABANA PT BRIDGE CONFIGURATION =========================== 
 PT_HPU_LAZY_MODE = 1
 PT_RECIPE_CACHE_PATH = 
 PT_CACHE_FOLDER_DELETE = 0
 PT_HPU_RECIPE_CACHE_CONFIG = 
 PT_HPU_MAX_COMPOUND_OP_SIZE = 9223372036854775807
 PT_HPU_LAZY_ACC_PAR_MODE = 1
 PT_HPU_ENABLE_REFINE_DYNAMIC_SHAPES = 0
---------------------------: System Configuration :---------------------------
Num CPU Cores : 160
CPU RAM       : 1056446876 KB
------------------------------------------------------------------------------


------------------------------------------------------------
Input: summarize: Introduction: The Strategic Arms Limitation Talks II (SALT II) treaty, signed on June 18, 1979, between the United States and the Soviet Union, marked a significant milestone in nuclear arms control efforts during the Cold War era. Building upon its predecessor, SALT I, the treaty aimed to curb the arms race and reduce the risk of nuclear conflict between the superpowers. Key Provisions: SALT II encompassed several crucial provisions. It placed limits on strategic offensive arms, including intercontinental ballistic missiles (ICBMs), submarine-launched ballistic missiles (SLBMs), and heavy bombers. The agreement specified the maximum number of deployed warheads and launchers each party could possess. Verification and Compliance: To ensure compliance, the treaty established comprehensive verification measures. This involved regular exchanges of data, on-site inspections, and monitoring activities by both nati

In [2]:
# To run additional inference examples, the jupyter notebook requires that the kernel be restarted.  this `exit()` command will restart the kernel and allow another infernece run.
exit()